# Data Manipulation

In [1]:
import os
import shutil

source_directory = r"C:\Users\AKAM\OneDrive\Desktop\Challenge\data_challenge\images\train\images"
destination_directory = r"C:\Users\AKAM\OneDrive\Desktop\Challenge\Self_Annotated_data_challenge\images"

batch_size = 200
file_counter = 0
batch_number = 1

os.makedirs(os.path.join(destination_directory, f"batch_{batch_number}"), exist_ok=True)

for filename in os.listdir(source_directory):
    if filename.endswith(".png"):  
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, f"batch_{batch_number}", filename)
        
        shutil.copy(source_path, destination_path)
        
        file_counter += 1
        if file_counter >= batch_size:
            file_counter = 0
            batch_number += 1
            os.makedirs(os.path.join(destination_directory, f"batch_{batch_number}"), exist_ok=True)


## Masks

In [3]:
import json
import os
from PIL import Image, ImageDraw

# Load JSON file
json_file_path = "Challenge_1st_trial.json"
output_dir = "output_masks"
os.makedirs(output_dir, exist_ok=True)

# Loop through annotations and create masks
with open(json_file_path, "r") as json_file:
    for line in json_file:
        annotation = json.loads(line)
        
        image_filename = annotation["filename"]
        image_width = annotation["size"]["width"]
        image_height = annotation["size"]["height"]
        
        mask = Image.new("L", (image_width, image_height), 0)
        draw = ImageDraw.Draw(mask)
        
        for shape in annotation["regions"]:
            class_name = shape["region_attributes"]["class"]
            
            if class_name == "1":  # Wealthy regions
                mask_value = 255
            elif class_name == "2":  # Unwealthy regions
                mask_value = 128
            else:
                continue
            
            x_points = shape["shape_attributes"]["all_points_x"]
            y_points = shape["shape_attributes"]["all_points_y"]
            polygon_points = list(zip(x_points, y_points))
            draw.polygon(polygon_points, fill=mask_value)
        
        mask_filename = os.path.splitext(image_filename)[0] + "_mask.png"
        mask.save(os.path.join(output_dir, mask_filename))

print("Masks generated successfully.")


In [5]:
import numpy as np
import cv2
import json

In [6]:
#reading json
f=open("Challenge_3rd_trial.json","r")
data=json.load(f)
#print(data)

In [7]:
data=data["_via_img_metadata"]
#print(data)

In [8]:
#path
img_dir=r"C:\Users\AKAM\OneDrive\Desktop\Challenge\Self_Annotated_data_challenge\images\batch_3"
mask_dir=r"C:\Users\AKAM\OneDrive\Desktop\Challenge\Self_Annotated_data_challenge\masks"

In [9]:
for key,value in data.items():
    filename=value["filename"]
    
    img_path=f'{img_dir}/{filename}'
    img=cv2.imread(img_path,cv2.IMREAD_COLOR)
    h,w,_=img.shape
    mask=np.zeros((h,w))
    regions=value["regions"]
#     print(value)
#     print(" ")
    for region in regions:
        shape_attributes=region["shape_attributes"]
        region_attributes=region["region_attributes"]
        x_points=shape_attributes["all_points_x"]
        y_points=shape_attributes["all_points_y"]
        
        contours=[]
        for x,y in zip(x_points,y_points):
            contours.append((x,y))
        contours=np.array(contours)
        if "1" in list(region_attributes["name"]):
            cv2.drawContours(mask,[contours],-1,255,-1)
        else:
            cv2.drawContours(mask,[contours],-1,128,-1)
#         print(region_attributes)
#         print(" ")
    mask=cv2.imwrite(f"{mask_dir}/{filename}",mask)
